# Image analysis

発現数に関する画像解析を可視化

In [25]:
import os
import numpy as np
from glob import glob
from skimage import io
from skimage.filters import threshold_otsu
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.color import label2rgb
from PIL import Image


def watershed_segmentation(binary_img, kernel_size, min_distance):
    distance = ndi.distance_transform_edt(binary_img)
    coords = peak_local_max(distance,
                            footprint=np.ones((kernel_size, kernel_size)),
                            labels=binary_img,
                            min_distance=min_distance)#ball(radius=kernel_size)
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndi.label(mask)
    labels = watershed(-distance, markers, mask=binary_img)
    return labels

def img_resize(image, resize):
    if len(image.shape) > 2:
        channel = image.shape[2]
        out = np.zeros((resize[0], resize[1], channel))
        for c in range(channel):
            image_c = Image.fromarray(image[:, :, c])
            if image_c.size[0] > resize[0] or image_c.size[1] > resize[1]:  # 縮小
                image_c = image_c.resize((resize[1], resize[0]))
            elif image_c.size[0] < resize[0] or image_c.size[1] < resize[1]:  # 拡大
                image_c = image_c.resize((resize[1], resize[0]), resample=Image.BICUBIC)
            out[:, :, c] = np.array(image_c)
    else:
        out = np.zeros((resize[0], resize[1]))
        image_c = Image.fromarray(image)
        if image_c.size[0] > resize[0] or image_c.size[1] > resize[1]:  # 縮小
            image_c = image_c.resize((resize[1], resize[0]))
        elif image_c.size[0] < resize[0] or image_c.size[1] < resize[1]:  # 拡大
            image_c = image_c.resize((resize[1], resize[0]), resample=Image.BICUBIC)
        out[:, :] = np.array(image_c)
    return out


def image_analysis(path, resize=(256,256), gt_mode=False):
    # settings
    ksize = 5
    sigma = 1.0
    truncate = ((ksize-1)/2-0.5)/sigma

    # load image
    img_raw = io.imread(path)
    img_raw_for_return = img_raw.copy()
    img_raw_for_return = (img_raw_for_return/65535)*255
    img_raw_for_return = img_raw_for_return.astype(np.uint8)
    if gt_mode:
        img_raw = img_resize(image=img_raw, resize=resize)
    assert img_raw.shape==resize, f'Shape not matched {img_raw.shape}:{resize}'

    # preprocess
    img = ndi.median_filter(img_raw, size=ksize)
    img = ndi.gaussian_filter(img, sigma=sigma, truncate=truncate)

    # binarize
    thresh = threshold_otsu(img)
    img_binary = img > thresh

    # segmentation
    img_label = watershed_segmentation(binary_img=img_binary, kernel_size=11, min_distance=11)
    img_label_color = label2rgb(img_label)*255
    img_label_color = img_label_color.astype(np.uint8)
    img_label = np.where(img_label>0,255,0).astype(np.uint8)
    return img_raw_for_return, img_label, img_label_color

exp_name = 'trial_wellplate_epoch100_batch28'
id_name = 'r01c06f06p01'#'r14c02f04p01'#'r01c06f06p01'
test_name = 'U2OS_48h'
save_dir = f'./visualize_image_analysis/{exp_name}/{test_name}/{id_name}'
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

true_root = f'/Users/morikura/Documents/Ozeki/pairedimagetoimagetranslation/results/wsb/{exp_name}/train_fold1_20240301-012459/test/loss/images/{id_name}/GT'
true_img_paths = sorted(glob(f"{true_root}/*.tif"))
print(len(true_img_paths))

wsb_root = f'/Users/morikura/Documents/Ozeki/pairedimagetoimagetranslation/results/wsb/{exp_name}/train_fold1_20240301-012459/test/loss/images/{id_name}/Predict'
wsb_img_paths = sorted(glob(f"{wsb_root}/*.tif"))
print(len(wsb_img_paths))


for img_path in true_img_paths:
    img_raw, img_label, img_label_color = image_analysis(path=img_path, gt_mode=True)
    filename = os.path.splitext(os.path.basename(img_path))[0]
    save_dir_each = f'{save_dir}/GT'
    if not os.path.exists(save_dir_each):
        os.makedirs(save_dir_each)
    io.imsave(f'{save_dir_each}/{filename}-raw.png', img_raw)
    io.imsave(f'{save_dir_each}/{filename}-label.png', img_label)
    io.imsave(f'{save_dir_each}/{filename}-label_color.png', img_label_color)

for img_path in wsb_img_paths:
    img_raw, img_label, img_label_color = image_analysis(path=img_path, gt_mode=True)
    filename = os.path.splitext(os.path.basename(img_path))[0]
    save_dir_each = f'{save_dir}/WSB'
    if not os.path.exists(save_dir_each):
        os.makedirs(save_dir_each)
    io.imsave(f'{save_dir_each}/{filename}-raw.png', img_raw)
    io.imsave(f'{save_dir_each}/{filename}-label.png', img_label)
    io.imsave(f'{save_dir_each}/{filename}-label_color.png', img_label_color)
    

./visualize_image_analysis/trial_wellplate_epoch100_batch28/U2OS_48h/r01c06f06p01
5
5


/var/folders/qf/bytm85n11x7b1snb8r2xqrg40000gp/T/ipykernel_54093/2282507498.py:100: UserWarning: ./visualize_image_analysis/trial_wellplate_epoch100_batch28/U2OS_48h/r01c06f06p01/GT/r01c06f06p01_channel_ch1-raw.png is a low contrast image
  io.imsave(f'{save_dir_each}/{filename}-raw.png', img_raw)
/var/folders/qf/bytm85n11x7b1snb8r2xqrg40000gp/T/ipykernel_54093/2282507498.py:110: UserWarning: ./visualize_image_analysis/trial_wellplate_epoch100_batch28/U2OS_48h/r01c06f06p01/WSB/r01c06f06p01_channel_ch1-raw.png is a low contrast image
  io.imsave(f'{save_dir_each}/{filename}-raw.png', img_raw)
/var/folders/qf/bytm85n11x7b1snb8r2xqrg40000gp/T/ipykernel_54093/2282507498.py:111: UserWarning: ./visualize_image_analysis/trial_wellplate_epoch100_batch28/U2OS_48h/r01c06f06p01/WSB/r01c06f06p01_channel_ch1-label.png is a low contrast image
  io.imsave(f'{save_dir_each}/{filename}-label.png', img_label)
/var/folders/qf/bytm85n11x7b1snb8r2xqrg40000gp/T/ipykernel_54093/2282507498.py:112: UserWarning